In [2]:
import networkx as nx
import os
import pandas as pd
import pickle
# import pickle5  # only Py 3.5, .6, .7

from datasketch import MinHash, MinHashLSH

#### Read LittleSis orgs

In [5]:
littlesis_orgs = {}

with open(os.getcwd() + '/Pickle/all_orgs.pickle', 'rb') as pkl:
    littlesis_orgs = pickle.load(pkl)
#     littlesis_orgs = pickle5.load(pkl)
    
len(littlesis_orgs)

78526

In [6]:
littlesis_df = pd.DataFrame.from_dict(list(littlesis_orgs.items()))
littlesis_df.rename(columns = {0: 'id', 1: 'org'}, inplace = True)
littlesis_df.head()

,id,org
0,1,Walmart
1,2,ExxonMobil
2,3,Chevron
3,4,General Motors Company
4,5,ConocoPhillips


#### Read bipartite graph

In [7]:
# bipartite_all = nx.read_gpickle('/home/lsheks/guidestar/bipartite-all010420.pickle')
bipartite_all = nx.read_gpickle(os.getcwd() + '/Pickle/bipartite-all010420.pickle')

bipartite_all.number_of_nodes()

4054832

In [8]:
bipartite_orgs = list(bipartite_all.nodes(data = 'Uppername'))  # nodes which don't have this key return None eg. (bipartite_id, None)
bipartite_orgs[: 5]

[(0, None), (1, None), (2, None), (3, None), (4, None)]

In [9]:
bipartite_orgs = [(bipartite_id, uppername) for bipartite_id, uppername in bipartite_orgs if uppername is not None]  # remove nodes where uppername is None i.e. not organisations
bipartite_orgs[: 5]

[(8388614, 'MISSIONS UNLIMITED'),
 (8388623, 'CORVALLIS-OSU SYMPHONY SOCIETY'),
 (8388624, 'OLD CHURCH SOCIETY INC'),
 (8388646, 'TVW INC'),
 (8388664, 'EDWARD F ARNOLD TRUST')]

In [10]:
len(bipartite_orgs)

696376

In [11]:
bipartite_df = pd.DataFrame(bipartite_orgs, columns = ['id', 'org'])
bipartite_df.head()

,id,org
0,8388614,MISSIONS UNLIMITED
1,8388623,CORVALLIS-OSU SYMPHONY SOCIETY
2,8388624,OLD CHURCH SOCIETY INC
3,8388646,TVW INC
4,8388664,EDWARD F ARNOLD TRUST


#### Clean org names

In [12]:
def cleanNames(df):
    if df['id'].dtype != 'int64':  # convert ids to int
        df['id'] = df['id'].apply(lambda x: int(x))
        
    # remove periods and commas, replace - with a single space, convert to upper case and strip leading and trailing whitespaces
    df['org'] = df['org'].apply(lambda x: x.replace('.', '').replace(',', '').replace('-', ' ').upper().strip())
    
    # other things to do (potentially):
    # 1. remove INC., LTD.
    # 2. remove special chars
    # also see Pyjanitor clean names: https://pyjanitor.readthedocs.io/reference/janitor.functions/janitor.clean_names.html
    
    return df

In [13]:
littlesis_df = cleanNames(littlesis_df)
littlesis_df.head()

,id,org
0,1,WALMART
1,2,EXXONMOBIL
2,3,CHEVRON
3,4,GENERAL MOTORS COMPANY
4,5,CONOCOPHILLIPS


In [14]:
bipartite_df = cleanNames(bipartite_df)
bipartite_df.head()

,id,org
0,8388614,MISSIONS UNLIMITED
1,8388623,CORVALLIS OSU SYMPHONY SOCIETY
2,8388624,OLD CHURCH SOCIETY INC
3,8388646,TVW INC
4,8388664,EDWARD F ARNOLD TRUST


In [12]:
clean_littlesis_orgs = littlesis_df.set_index('id')['org'].to_dict()

# max_num selects the bipartite subset size, select the second option to use the entire subset
max_num = len(bipartite_df)
# max_num = bipartite_df.shape[0]
clean_bipartite_orgs = bipartite_df.set_index('id')['org'][: max_num].to_dict()
clean_bipartite_orgs_rev = {clean_bipartite_orgs[i]: i for i in clean_bipartite_orgs}

# print(len(clean_littlesis_orgs), len(clean_bipartite_orgs), len(clean_bipartite_orgs_rev))

In [13]:
bip_df=pd.DataFrame.from_dict(clean_bipartite_orgs, orient='index', columns=['name'])
ls_df=pd.DataFrame.from_dict(clean_littlesis_orgs, orient='index', columns=['name'])
rev=nx.get_node_attributes(bipartite_all, 'total_revenue')
bip_df['total_revenue'] = bip_df.index.map(rev)

bip_df.head()

,name,total_revenue
9437186,STYLING FOR PINK,NaN
7340035,FRATERNAL ORDER OF EAGLES,122338.0
7360279,VALLEY COUNCIL CREDIT UNION INC,185281.0
8219211,VETERANS OF FOREIGN WARS OF THE UNITED STATES ...,592870.0
8388614,MISSIONS UNLIMITED,995008.0


#### Hash

In [14]:
remaining_bipartite = len(clean_bipartite_orgs)
count, rep_count, reps = 0, 0, set()
lsh = MinHashLSH(threshold = 0.5, num_perm = 128)

for org_id in clean_bipartite_orgs:
    p = MinHash(num_perm = 128)
    
    org = clean_bipartite_orgs[org_id]
    
    for org_split in org.split():
        p.update(org_split.encode('utf-8'))
        
    try:
        lsh.insert(org, p)
        count += 1
        if count % 10000 == 0:
            print('Inserting', count)  #, 'out of', remaining_bipartite)
    except:
#         print(org)
        rep_count += 1
        reps.add(org)
    
# print('Repeated orgs:', rep_count)
# print(reps)

Inserting 10000
Inserting 20000
Inserting 30000
Inserting 40000
Inserting 50000
Inserting 60000
Inserting 70000
Inserting 80000
Inserting 90000
Inserting 100000
Inserting 110000
Inserting 120000
Inserting 130000
Inserting 140000
Inserting 150000
Inserting 160000
Inserting 170000
Inserting 180000
Inserting 190000
Inserting 200000
Inserting 210000
Inserting 220000
Inserting 230000
Inserting 240000
Inserting 250000
Inserting 260000
Inserting 270000
Inserting 280000
Inserting 290000
Inserting 300000
Inserting 310000
Inserting 320000
Inserting 330000
Inserting 340000
Inserting 350000
Inserting 360000
Inserting 370000
Inserting 380000
Inserting 390000
Inserting 400000
Inserting 410000
Inserting 420000
Inserting 430000
Inserting 440000
Inserting 450000
Inserting 460000
Inserting 470000
Inserting 480000
Inserting 490000
Inserting 500000
Inserting 510000
Inserting 520000
Inserting 530000
Inserting 540000
Inserting 550000
Inserting 560000
Inserting 570000
Inserting 580000
Inserting 590000
Insert

#### Compare org names

In [15]:
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_kernels

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(analyzer='word')
vec.fit(list(bip_df['name']))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [17]:
def org_match(row_ls, df_bipartite):
    name=row_ls['name']
    row_ls_index=row_ls.index
    dist=[]
    stopset=['OF','THE','AND','&','AT','INC','INCORPORATED']
    target=" ".join([i for i in word_tokenize(name) if i not in stopset])


    original=[]
    candidate=[]
    names=[]
    
    p=MinHash(num_perm=128)
    for ww in name.split():
        p.update(ww.encode('utf-8'))
    result = lsh.query(p)

    for j in result:
        original.append(j)
        candidate.append(" ".join([i for i in word_tokenize(j) if i not in stopset]))
        names.append(name)

    #play with ngram range

    distance=0
    match=''
    if len(candidate)>0:
        distances=pairwise_kernels(vec.transform([target]),vec.transform(candidate),metric='cosine')

        df=pd.DataFrame({'Target':pd.Series(names),
                                 'Match':pd.Series(candidate),
                             'Original Match':pd.Series(original),
                                'Cosine Distance':pd.Series(list(distances.flatten()))})
        distance=df.loc[df['Cosine Distance'].idxmax()]['Cosine Distance']

    if distance > 0.01:
        match = df.loc[df['Cosine Distance'].idxmax()]['Original Match']
        distance=df.loc[df['Cosine Distance'].idxmax()]['Cosine Distance']
        dist=distance
        indexes =df_bipartite[df_bipartite['name'] == match].index.tolist()
        if (len(indexes) == 1) or (len(set(indexes)) == 1):
            index = int(indexes[0])
            row_ls['match_id']=index
        elif len(indexes) > 1:
            max_rev = -1
            max_idx = 0
            for idx in indexes:
                if df_bipartite.loc[idx, 'total_revenue'] > max_rev:
                    max_rev = df_bipartite.loc[idx, 'total_revenue']
                    max_idx = idx
            row_ls['match_id']=max_idx
    else:
        print('no match for '+name)
        row_ls['match_id']=None
        dist=0


    return row_ls['match_id'],dist, match

In [18]:
ls_df=ls_df.sort_index()
len(ls_df)

78526

In [19]:
df1=ls_df.apply(org_match,df_bipartite=bip_df,axis=1)
ls_df['match']=df1
ls_df['gs_match_id']=ls_df['match'].apply(lambda x: x[0] if len(x)>2 else np.nan)
ls_df['match_name']=ls_df['match'].apply(lambda x: x[2] if len(x)>2 else np.nan)
ls_df['match_value']=ls_df['match'].apply(lambda x: x[1] if len(x)>2 else np.nan)
ls_df=ls_df.drop('match', axis=1)

no match for WALMART
no match for WALMART
no match for EXXONMOBIL
no match for CHEVRON
no match for CONOCOPHILLIPS
no match for GENERAL ELECTRIC COMPANY
no match for FORD MOTOR COMPANY
no match for CITIGROUP INC
no match for AT&T INC
no match for MCKESSON
no match for THE PROCTER & GAMBLE COMPANY
no match for AMERISOURCEBERGEN
no match for COSTCO WHOLESALE CORPORATION
no match for MERRILL LYNCH PIERCE FENNER & SMITH INCORPORATED
no match for ANTHEM


KeyboardInterrupt: 

In [ ]:
ls_df.to_csv('matched_littlesis_bipartite171120.csv')